In [52]:
class Node:
    def __init__(self,data):
        self.data = data
        self.left = None
        self.right = None
        self.parent = None
        self.balance = 0
        self.height = 1
        self.flag = False
        
    def __str__(self):
        s = str(self.data) 
        s+= "  h:"+str(self.height)
        s+= " b:"+str(self.balance)
        return s
    
    def insert(self,node,root):
        if node.data <= self.data: 
            if self.left is None:  
                self.left = node
                node.parent = self
            else:
                self.left.insert(node,root)
        else:
            if self.right is None:  
                self.right = node
                node.parent = self
            else:
                self.right.insert(node,root)
                
        return self.mbalance(root)
        

    def rotate_left(self,node,root):
        parent = node.parent
        a = node.right
        b = node
        s2 = a.left
        
        b.right = s2
        if s2 is not None: s2.parent = b
            
        a.left = b
        b.parent = a
        
        b.mbalance(root)
        if self.flag is not True:
            a.mbalance(root)
        
        if parent is None:
            a.parent = None
            return a
        elif parent.left == b: parent.left = a
        elif parent.right == b: parent.right = a
        a.parent = parent
        
        
        return root
    
    def rotate_right(self,node,root):
        parent = node.parent
        a = node
        b = node.left
        s2 = b.right
        
        a.left = s2
        if s2 is not None: s2.parent = a
            
        b.right = a
        a.parent = b
        
        a.mbalance(root)
        if self.flag is not True:
            b.mbalance(root)
        
        if parent is None:
            b.parent = None
            return b
        elif parent.left == a: parent.left = b
        elif parent.right == a: parent.right = b
        b.parent = parent
        
        
        return root
        
    def mbalance(self,root):
        height_left = 0 if self.left is None else self.left.height
        height_right = 0 if self.right is None else self.right.height
        self.height = max(height_left+1,height_right+1)
        self.balance = height_left - height_right
#         print(self)
        
        if self.balance==-2 and self.right.balance<=0:
            root = self.rotate_left(self,root)
            
        elif self.balance==-2 and self.right.balance==1:
            self.flag = True
            root = self.rotate_right(self.right,root)
            self.flag = False
            root = self.rotate_left(self,root)
            
        elif self.balance==2 and self.left.balance == -1:
            self.flag = True
            root = self.rotate_left(self.left,root)
            self.flag = False
            root = self.rotate_right(self,root)
            
        elif self.balance==2 and self.left.balance>=0:
            root = self.rotate_right(self,root)
            
        return root
    
        
class AVL:
    def __init__(self):
        self.root = None
        self.flag = False
    
    def __print_node(self,node,sp):
        s = sp + node.__str__() + "\n"
        if node.left is not None: s+= self.__print_node(node.left,sp+" l:")
        if node.right is not None: s+= self.__print_node(node.right,sp+" r:")
        return s
        
    def __str__(self):
        if self.root is None: return "";
        else:  return self.__print_node(self.root,"")
        
    def insert(self,data):
        node = Node(data)
        if self.root is None:
            self.root = node
        else:
            self.root = self.root.insert(node,self.root)
#         print()
            
            
            
    def __search_parent_node(self,data):
        if self.root is None:
            return None, None
        parent = None
        node = self.root
        while True:
            if data == node.data:
                return parent, node
            elif data < node.data: #izquierda
                parent = node
                node = node.left
            elif data > node.data: #derecha
                parent = node
                node = node.right
            if node is None:
                return None,None
    
    def __remove(self,parent,node):
        if node.left is None and node.right is None:
#             print('   Caso 1')
            if parent is None: self.root = None
            elif parent.left == node:  
                parent.left = None
                if self.flag is not True:
                    self.retroceso(parent)
            elif parent.right == node: 
                parent.right = None
                if self.flag is not True:
                    self.retroceso(parent)
            
        elif (node.left is None and node.right is not None) or (node.left is not None and node.right is None):
#             print('   Caso 2')
            if parent is None:
                if node.left is not None: 
                    self.root = node.left
                elif node.right is not None: 
                    self.root = node.right
                self.root.parent = None
            elif parent.left == node:
                if node.left is not None: parent.left = node.left
                elif node.right is not None: parent.left = node.right
                parent.left.parent = parent
                if self.flag is not True:
                    self.retroceso(parent)
            elif parent.right == node:
                if node.left is not None: parent.right = node.left
                elif node.right is not None: parent.right = node.right
                parent.right.parent = parent
                if self.flag is not True:
                    self.retroceso(parent)
            
        elif node.left is not None and node.right is not None:
#             print('   Caso 3')
            parent2 = node
            node2 = node.right
            while node2.left is not None:
                parent2 = node2
                node2 = node2.left
            if parent2 is node:
                aux = node2
            else:
                aux = parent2
            self.flag = True
            self.__remove(parent2,node2)
            self.flag = False
            node2.left = node.left
            node2.right = node.right
            node2.parent = parent
            if node2.left is not None:
                node2.left.parent = node2
            if node2.right is not None:
                node2.right.parent = node2
            
            if parent is None: 
                self.root = node2
                self.retroceso(aux)
            elif parent.left == node:  
                parent.left = node2
                self.retroceso(aux)
            elif parent.right == node: 
                parent.right = node2
                self.retroceso(aux)
        
        
    def remove(self,data):
        print('Remove:')
        parent,node = self.__search_parent_node(data)
        if node is None:
            return
        self.__remove(parent,node)
        print('   Delete',node)
        
        
    def retroceso(self,parent):
        if parent is not None:
            aux = parent
            while aux.parent is not None:
                self.root = aux.mbalance(self.root)
                aux = aux.parent
            if self.root is not None:
                self.root = self.root.mbalance(self.root)
            
    

In [53]:
tree = AVL()
tree.insert(90)
tree.insert(89)
tree.insert(88)
tree.insert(87)
tree.insert(86)
tree.insert(85)
tree.insert(84)
tree.insert(83)
tree.insert(82)
tree.insert(81)
tree.insert(80)
tree.insert(79)
tree.insert(78)
tree.insert(100)
tree.insert(91)
tree.insert(92)
tree.insert(1)
tree.insert(5)
tree.insert(4)
tree.insert(6)
tree.insert(7)
tree.insert(20)
tree.insert(15)
tree.insert(16)
tree.insert(17)
tree.insert(18)
tree.insert(19)
tree.insert(3)


print("Arbolito")
print(tree)

tree.remove(90)
tree.remove(83)
tree.remove(20)
tree.remove(15)
tree.remove(3)
tree.remove(91)
tree.remove(85)
tree.remove(10000)
tree.remove(84)
tree.remove(1)
tree.remove(7)
tree.remove(5)
tree.remove(100)




print()
print("Arbolito")
print(tree)

print()
print(tree.root.parent)

Arbolito
83  h:6 b:1
 l:7  h:5 b:-1
 l: l:5  h:3 b:1
 l: l: l:3  h:2 b:0
 l: l: l: l:1  h:1 b:0
 l: l: l: r:4  h:1 b:0
 l: l: r:6  h:1 b:0
 l: r:20  h:4 b:0
 l: r: l:17  h:3 b:0
 l: r: l: l:16  h:2 b:1
 l: r: l: l: l:15  h:1 b:0
 l: r: l: r:18  h:2 b:-1
 l: r: l: r: r:19  h:1 b:0
 l: r: r:79  h:3 b:-1
 l: r: r: l:78  h:1 b:0
 l: r: r: r:81  h:2 b:0
 l: r: r: r: l:80  h:1 b:0
 l: r: r: r: r:82  h:1 b:0
 r:87  h:4 b:-1
 r: l:85  h:2 b:0
 r: l: l:84  h:1 b:0
 r: l: r:86  h:1 b:0
 r: r:91  h:3 b:0
 r: r: l:89  h:2 b:0
 r: r: l: l:88  h:1 b:0
 r: r: l: r:90  h:1 b:0
 r: r: r:100  h:2 b:1
 r: r: r: l:92  h:1 b:0

Remove:
   Delete 90  h:1 b:0
Remove:
   Delete 83  h:6 b:1
Remove:
   Delete 20  h:4 b:0
Remove:
   Delete 15  h:1 b:0
Remove:
   Delete 3  h:2 b:0
Remove:
   Delete 91  h:3 b:0
Remove:
   Delete 85  h:2 b:-1
Remove:
Remove:
   Delete 84  h:4 b:0
Remove:
   Delete 1  h:1 b:0
Remove:
   Delete 7  h:4 b:-1
Remove:
   Delete 5  h:2 b:0
Remove:
   Delete 100  h:1 b:0

Arbolito
78  h:5 